<a href="https://colab.research.google.com/github/josipbencic/kaggle/blob/master/predict_future_sales_kristijan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive

ROOT = "/content/drive"     # default location for the drive
PROJECT_PATH = "/content/drive/My Drive/kaggle_tmp"

drive.mount(ROOT)           # we mount the google drive at /content/drive

GIT_TOKEN = "097309d50a5b76d973c1011b2194720f119234c8"
GIT_USERNAME = "josipbencic"
GIT_REPOSITORY = "kaggle"

GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"

!git clone "{GIT_PATH}" ./temp      # clone github repository to temp folder
!cp -R ./temp/* "{PROJECT_PATH}"    # move all files/folders in temp folder to folder defined in project path
!rm -rf ./temp                      # remove all the files/folders in temp folder
#!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./   # use remote sync to copy from google drive to local runtime google colab
                                                     # but exclude data folder                                          
!rsync -aP "{PROJECT_PATH}"/* ./  

!pip install category_encoders
!pip install pycuda
!pip install scikit-cuda
!pip install tensorflow

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Cloning into './temp'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 59 (delta 25), reused 22 (delta 4), pack-reused 0
Unpacking objects: 100% (59/59), done.
sending incremental file list
README.md
            605 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=11/12)
environment.yml
            211 100%  206.05kB/s    0:00:00 (xfr#2, to-chk=10/12)
predict_future

In [1]:
## Inspect collab machine
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=fdd954aa6d6ec22da85405cd9e28c81f94c80fd062fd9d6eb08108ddac5f5033
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 159.0 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [1]:
%%time
from typing import List, Set, Dict, Tuple, Optional
import argparse
import sys
import os
import csv
import time
import math
from datetime import date
from math import floor

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import category_encoders as ce
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse as mse
#from statsmodels.multivariate.pca import PCA
from sklearn.decomposition import PCA

import pycuda.autoinit
import pycuda.gpuarray as gpuarray
import numpy as np
import skcuda.linalg as linalg
from skcuda.linalg import PCA as cuPCA


file_prefix = './data/'

### Load the datasets from disk
itemCategories=pd.read_csv(file_prefix + 'item_categories.csv')
shops=pd.read_csv(file_prefix + 'shops.csv')
test=pd.read_csv(file_prefix + 'test.csv')
sales_original = pd.read_csv(file_prefix + 'sales_train.csv')

items = pd.read_csv(file_prefix + 'items.csv')
items = items.drop('item_name', axis = 1)
items.set_index('item_id', inplace = True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ModuleNotFoundError: ignored

In [0]:
%%time
SPAN = 5
sales = sales_original.copy()

def build(lo, hi): # [lo, hi)
  data = sales[
      (sales.date_block_num < hi) & (sales.date_block_num >= lo)
    ].groupby(['date_block_num', 'shop_id', 'item_id']).agg(
      sold = pd.NamedAgg(column = 'item_cnt_day', aggfunc = 'sum'),
      price = pd.NamedAgg(column = 'item_price', aggfunc = 'first')
    )
  data.reset_index(drop = False, inplace = True)
  
  ## Append item_id category variable from the other dataset
  data.set_index('item_id', drop = True, inplace = True)
  data = data.join(items, on = 'item_id')
  data.reset_index(inplace = True, drop = False)


  ### append historic data for up to SPAN months before
  aggs = data.copy()
  aggs.drop(inplace = True, columns = ['item_category_id'])
  aggs.rename(columns = {'date_block_num': 'date_block_before'}, inplace = True)
  aggs.set_index(
      ['item_id', 'shop_id', 'date_block_before'], inplace = True, drop = True)

  for it in range(1, SPAN + 1):
    data['date_block_before'] = data.date_block_num - it
    ## Here is where we filter for the first month/s to always have all features
    ## (some features are computed back in time, eg. last month's price)
    data = data[data.date_block_before >= 0]
    data = data.join(
        aggs,
        on = ['item_id', 'shop_id', 'date_block_before'],
        rsuffix = ('_' + str(it))
    )
    data.drop('date_block_before', axis = 1, inplace = True)

    #### Some combinations of shop-item have 0 sales in some months
    ## those items that haven't had any sales in the given slots are assigned 0
    ## instead of NaN
    data['sold_' + str(it)] = data['sold_' + str(it)].fillna(0)
    ## If the price wasn't present at the time:
    ##   - first, fill backwards in time the last price (works with multiple prices)
    ##   - then fill front
    data['price_' + str(it)] = data['price_' + str(it)].fillna(method = 'backfill')
    data['price_' + str(it)] = data['price_' + str(it)].fillna(method = 'ffill')
    data['price_' + str(it) + '_log'] = np.log(data['price_' + str(it)].values)
    data = data.drop('price_' + str(it), axis = 1)
  data['price_log'] = np.log(data['price'].values)

  data = data.drop(['date_block_num'], axis = 1)
  data.reset_index(inplace = True, drop = True)
  return data

In [0]:
%%time
## Separate and clean the data

df = build(0, 32)
cv = build(32 - SPAN, 33)

### Encoding

## One-hot-encoding for item_id, shop_id, item_category_id
## drop_invariant has to be True
##    it means the constant columns will be dropped
##    this will prevent the normal equations from failing - the matrix won't
##    be singular
enc = ce.BinaryEncoder(cols=['item_id', 'shop_id', 'item_category_id'],
                           drop_invariant = True)
enc.fit(df.loc[:, df.columns != 'sold'], df.sold)
X_enc = enc.transform(df.loc[:, df.columns != 'sold'])
Y_enc = df.sold


X_cv_enc = enc.transform(cv.loc[:, cv.columns != 'sold'])
Y_cv_enc = cv.sold


## PCA
#pca = cuPCA(epsilon = 1e-2)
#X_gpu = gpuarray.GPUArray(X_enc.shape, np.float64, order="F")
#X_gpu.set(X_enc.values)
#Tmp_gpu = pca.fit_transform(X_gpu)
#X = Tmp_gpu.get()

pca = PCA(n_components = 4)
pca.fit(X_enc)


X = pca.transform(X_enc)
X_cv = pca.transform(X_cv_enc)

In [0]:
def weight_variable(shape, w=0.1):
  initial = tf.truncated_normal(shape, stddev=w)
  return tf.Variable(initial)

def bias_variable(shape, w=0.1):
  initial = tf.constant(w, shape=shape)
  return tf.Variable(initial)

In [0]:
indim=trainx.shape[1]
l1=10
outp=1
W1=weight_variable([indim,l1])
W2=weight_variable([l1,outp])

b1=bias_variable([l1])
b2=bias_variable([outp])

x=tf.placeholder(tf.float32,[None,indim],name="input")
y=tf.placeholder(tf.float32,[None,outp],name="output")

trainep=1000

model1=tf.nn.relu(tf.matmul(x,W1)+b1)
model2=tf.sigmoid(tf.matmul(model1,W2)+b2)

loss=tf.reduce_sum(tf.square(model2-y))

train=tf.train.GradientDescentOptimizer(1).minimize(loss)


init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

for i in range (trainep):       #ustekas input i output i pustis da trenira
  sess.run(train,feed_dict={x:X, y:Y_enc })
  l=sess.run(loss,feed_dict={x:X, y:Y_enc })
  print(l)
sess.close()




